In [ ]:
### Import Libraries.

library("tidyverse")
library("liana")
library("Seurat")
library("scater")
library("ComplexHeatmap")
library("circlize")
library("pheatmap")
library("RColorBrewer")
library("iTALK")
library("cowplot")
library("nichenetr")
library("readxl")
library("presto")
library("future")

options(future.globals.maxSize = Inf)

In [ ]:
show_resources()
show_methods()

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- readRDS("Seurat.rds")
sce <- as.SingleCellExperiment(Seurat, assay = "RNA")

In [ ]:
### Balanced Subsampling (100,000 cells). (Optional)

set.seed(123)

n_cells <- 100000
cell_df <- tibble(cell = colnames(Seurat), subcluster = Seurat$Cluster_Column)

sub_counts <- table(cell_df$subcluster)
sampling_numbers <- round(n_cells * sub_counts / sum(sub_counts))

diff <- n_cells - sum(sampling_numbers)
if (diff != 0) {
  adjust <- sample(names(sampling_numbers), abs(diff))
  sampling_numbers[adjust] <- sampling_numbers[adjust] + sign(diff)
}

sampled_cells <- unlist(lapply(names(sampling_numbers), function(sub) {
  sample(cell_df$cell[cell_df$subcluster == sub],
         sampling_numbers[sub])
}))

Seurat_sub <- subset(Seurat, cells = sampled_cells)

In [ ]:
### LIANA Multi-method Inference.

liana <- liana_wrap(
  sce = sce,
  method = c("cytotalk", "connectome", "logfc", "natmi", "sca"),
  resource = "Consensus",
  idents_col = "Cluster_Column",
  assay.type = "logcounts",
  min_cells = 10,
  expr_prop = 0.4,
  permutation.params = list(nperms = 100), ### Optimize permutations.
  verbose = TRUE,
  parallelize = TRUE,
  workers = 50
)

liana$call_cellchat <- NULL
liana$call_italk <- NULL

In [ ]:
### Aggregate LIANA Scores.

liana_agg <- rank_aggregate(liana)

In [ ]:
### Long Format & Ligand Filtering.

scores_long <- liana_agg %>%
  pivot_longer(cols = -c(source, target, ligand.complex, receptor.complex),
               names_to = "score", values_to = "value")

scores_sub <- scores_long %>%
  filter(score == "magnitude_rank", source != target)

ligands <- unique(scores_sub$ligand.complex)

In [ ]:
##t# Filter Interactions (q ≤ 0.05).

liana_trunc <- liana_agg %>% filter(magnitude_rank <= 0.05)

In [ ]:
### Dotplot Example.

p_dot <- liana_agg %>%
  liana_dotplot(
    source_groups = "Cluster_1",
    target_groups = "Cluster_2",
    specificity    = "natmi.edge_specificity",
    magnitude      = "sca.LRscore",
    ntop = 25
  )

In [ ]:
### Density of Method Scores.

scores_long %>%
  filter(!str_detect(score, "rank")) %>%
  ggplot(aes(value, fill = score)) +
  geom_density(alpha = 0.5) +
  facet_wrap(~score, scales = "free", ncol = 3) +
  theme_bw()

In [ ]:
### Heatmap for Ligand of Interest.

genes_of_interest <- "SPP1"

filtered_df <- liana_trunc %>%
  filter(ligand.complex %in% genes_of_interest |
         receptor.complex %in% genes_of_interest)

heat_data <- filtered_df %>%
  select(source, target, magnitude_rank) %>%
  group_by(source, target) %>%
  summarise(score = mean(magnitude_rank), .groups = "drop") %>%
  complete(source, target, fill = list(score = 0))

ggplot(heat_data, aes(target, source, fill = score)) +
  geom_tile(color = "white") +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Sender–Receiver Interaction (SPP1)",
       x = "Receiver", y = "Sender")

In [ ]:
### Chord & Heatmap (global).

chord_freq(liana_agg,
           source_groups = "Cluster_1",
           target_groups = "Cluster_2")

heat_freq(liana_trunc)

In [ ]:
### LIANA + NicheNet Integration.

expression <- t(as.matrix(GetAssayData(Seurat, layer = "data")))
metadata   <- Seurat@meta.data %>% rownames_to_column("Cell_ID")

ligand_target_matrix <- readRDS(
  url("https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final.rds")
)

results <- liana_agg %>%
  filter(source == "Cluster_1",
         target == "Cluster_2") %>%
  rename(ligand = ligand.complex,
         receptor = receptor.complex)

top_n <- results %>%
  arrange(magnitude_rank) %>%
  slice_head(n = 100) %>%
  mutate(id = fct_inorder(paste0(ligand, " -> ", receptor)))

In [ ]:
### Target Gene Set for NicheNet.

ligands <- c("Feature_1","Feature_2","Feature_3","Feature_4") %>%
  intersect(colnames(ligand_target_matrix))

background_genes <- expression[metadata$Cell_ID[metadata$Supercluster ==
                                                  "Cluster_1"], ] %>%
  apply(2, function(x) log2(10*(2**x - 1) + 1)) %>%
  .[. >= 4] %>% names()

genes <- read_xlsx("cluster_column_marker_genes.xlsx") %>%
  filter(p_val_adj < 0.05, pct.1 > 0.2) %>%
  pull(gene) %>%
  intersect(rownames(ligand_target_matrix))

In [ ]:
### Ligand Activity.

nichenet_activities <- predict_ligand_activities(
  geneset = genes,
  background_expressed_genes = background_genes,
  ligand_target_matrix = ligand_target_matrix,
  potential_ligands = ligands
)

vis <- top_n %>%
  inner_join(nichenet_activities, by = c("ligand" = "test_ligand")) %>%
  arrange(pearson) %>%
  mutate(ligand = fct_inorder(ligand))

In [ ]:
### LIANA (Receptor Heatmap) + NicheNet (Ligand Scores).

p_nn <- vis %>%
  group_by(ligand) %>%
  summarise(pearson = mean(pearson)) %>%
  ggplot(aes(ligand, pearson)) +
  geom_col() +
  ggtitle("NicheNet ligand activity") +
  theme_cowplot() +
  theme(axis.text.x = element_text(angle = 60, hjust = 1))

p_liana <- vis %>%
  ggplot(aes(receptor, ligand, fill = magnitude_rank)) +
  geom_tile() +
  theme_cowplot() +
  ggtitle("LIANA receptor engagement") +
  theme(axis.text.x = element_text(angle = 60, hjust = 1))

plot_grid(p_liana, p_nn, align = "h",
          nrow = 1, rel_widths = c(0.8, 0.3))